### Build the dataframe from the raw files

In [1]:
# Import required packages
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [2]:
def parse_xml(xml_file):
    global empty_counter

    # Load the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Define the namespace
    namespace = {
        'atom': 'http://www.w3.org/2005/Atom',
        'nitf': 'http://iptc.org/std/NITF/2006-10-18/',
        'dc': 'http://purl.org/dc/elements/1.1/'
    }

    # Extract the ID
    nexis_id = root.find('.//atom:id', namespace).text
    nexis_id = nexis_id.split('Item:')[1]
    nexis_id = nexis_id.replace("-", "")

    # Extract the title
    title = root.find('.//atom:title', namespace).text

    # Extract the published date
    published_date = root.find('.//atom:published', namespace).text
    published_date = published_date.split('T')[0]

    # Extract the publisher
    publisher = root.find('.//metadata/publicationInfo/publicationName').text


    # Extract the body text
    body_text = root.find('.//nitf:body.content//bodyText//p', namespace)

    # Extract all body paragraphs and join them into a single line
    body_paragraphs = []
    body_text_element = root.find('.//nitf:body.content//bodyText', namespace)
    if body_text_element is not None:
        for p_element in body_text_element.findall('.//p', namespace):
            if p_element.text:
                body_paragraphs.append(p_element.text.strip())
    text = ' '.join(body_paragraphs)

    # Check for empty text bodies and return
    if text == '':
        #print('No text body in file ' + file_path)
        empty_counter += 1
        return

    # Create a dictionary with the extracted values
    data_dict = {
        'nexis_id': [nexis_id],
        'title': [title],
        'publication_date': [published_date],
        'publisher': [publisher],
        'text': [text]
    }

    df = pd.DataFrame(data_dict)
    return df

In [3]:
# Root Directory of the data
folder_path = 'c:\\Friendcloud\\_University\\_Masterarbeit\\Data\\Nexis\\'

# Initialize empty counter
empty_counter = 0

# Go through all subdirectories and fill raw_data with the data
raw_data = []
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xml'):
            try:
                file_path = os.path.join(root, file)
                data = parse_xml(file_path)
                raw_data.append(data)
            except:
                print('Error in file ' + file_path)

# Build pandas DataFrame
df_raw = pd.concat(raw_data, ignore_index=True)
print('Number of articles removed: ' + str(empty_counter))
print('Number of articles accepted: ' + str(len(df_raw.index)))

Number of articles removed: 90
Number of articles accepted: 6779


In [4]:
df_raw

,nexis_id,title,publication_date,publisher,text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...,...
6774,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
6775,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
6776,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
6777,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [5]:
df_raw.to_pickle("../Data/df_raw.pkl")

### Data preprocessing

In [6]:
df_processed = pd.read_pickle("../Data/df_raw.pkl")

In [7]:
# Add year of publication to dataset
df_processed['publication_date'] = pd.to_datetime(df_processed['publication_date'], errors='coerce')
df_processed['publication_year'] = df_processed['publication_date'].dt.year
df_processed = df_processed[['nexis_id', 'title', 'publication_date', 'publication_year', 'publisher', 'text']]

In [8]:
# Remove articles with duplicate id
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='nexis_id', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 3


In [9]:
# Find examples of articles with duplicate text
df_processed[df_processed.duplicated(subset='text', keep=False)].sample(10)

,nexis_id,title,publication_date,publication_year,publisher,text
6024,61GC73X1DY4BY2G00000000,«Wir brauchen keine Lehren»,2020-12-09,2020,Toggenburger Tagblatt,Die Entwicklung der Coronasituation bereitet d...
2259,4C5VDP90TWH7K24B0000000,"Vier, die ausziehen, um Europa zu erobern",2004-04-17,2004,Frankfurter Rundschau,VON JUTTA OCHS UND CLAUS-JÜRGEN GÖPFERT Patric...
2997,658T7VY1DYM1G1CF0000000,None,2022-04-20,2022,Langenthaler Tagblatt,Serie: Was essen wir morgen? Fleisch aus dem L...
6694,5SF7W841F16T20T30000000,"Einen sauberen Burger, bitte!",2018-05-30,2018,Wiler Zeitung,Lebensmittel Fleischkonsum ohne schlechtes Ge...
5369,5WTKBKR1F16T24B60000000,Schlupfloch Schweizer Markt,2019-08-15,2019,Tagblatt für den Kanton Thurgau,«Hormonfleisch» ist in der EU verboten - nicht...
209,5WW9W861F0CCR52B0000000,"""Clean Meat"" - Der Burger kommt bald aus dem L...",2019-07-14,2019,AWP Premium Swiss News (German),Madrid/Zürich (awp/awp/sda/reu) - Als das nied...
5380,64SPSBG1JBN970YJ0000000,«Zuchtfleisch ist nicht ökologischer»,2022-02-15,2022,Tagblatt für den Kanton Thurgau,"Hafer- oder Kuhmilch, Import-Bananen oder Trei..."
6379,68VM2GG1JCSCJ2HG0000000,Warum Hornissen dieses Jahr viel gesichtet werden,2023-08-03,2023,Neue Westfälische,Sie ist auffallend gelb gestreift und beim Fli...
6548,608BJ6V1JDRK82WN0000000,Redefine Meat,2020-07-02,2020,WELT ONLINE (Deutsch),"""Sollen wir heute grillen? Alles klar, ich sch..."
2699,5SBJW171F13R83B50000000,Angst vor Resistenzen,2018-05-17,2018,Idsteiner Zeitung (Germany),"BIELEFELD (dpa). Antibiotikaresistenzen, gente..."


In [10]:
# Remove articles with duplicate text
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='text', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 1278


In [11]:
# Remove articles with length less than 100 characters
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['text'].str.len() > 100]
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 73


In [12]:
# Reindex the dataframe
df_processed = df_processed.reset_index(drop=True)

In [13]:
df_processed

,nexis_id,title,publication_date,publication_year,publisher,text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...,...,...
5420,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
5421,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
5422,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
5423,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [14]:
df_processed.to_pickle("../Data/df.pkl")